# Comparison of pMELTS to experimental clinopyroxenes

This notebook uses pMELTS to generate DataFrames and excel spreadsheets of predicted clinopyroxene compositions based on an experimental composition, pressure and temperature. To run this code please first install pyMELTScalc and download the MELTS files from https://magmasource.caltech.edu/gitlist/MELTS_Matlab.git/

In [1]:
#!pip install "https://github.com/gleesonm1/pyMELTScalc/archive/refs/tags/v0.1.9.zip" 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyMELTScalc as M

import sys
# use this line to inform the code where the MELTS files are stored
sys.path.append(r'H:\My Drive\2022\GitHub\pyMELTScalc\MELTS')

## First calculate the clinopyroxene compositions predicted for experiments used in the Neave and Putirka (2017) calibration

In [3]:
Data = pd.read_excel("NeaveExps.xlsx")

In [4]:
Res = M.equilibrate_multi(Model = "pMELTS", P_bar = Data['P_kbar']*1000, T_C = Data['T_K']-273.15, 
                          bulk = Data, fO2_buffer = "NNO")

  0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
Results = pd.DataFrame(columns = Res['0']['liquid1'].columns, 
                       data = np.zeros((len(Data['SiO2_Liq']), len(Res['0']['liquid1'].columns))))

for i in Res:
    if 'clinopyroxene1' in list(Res[i].keys()):
        Results.iloc[int(i), :] = Res[i]['clinopyroxene1']
        
Results['FeOt'] = Results['FeO'] + Results['Fe2O3']*71.844/(159.69/2)
Results = Results.add_suffix('_MELTS_Cpx')

for r in Results:
    Data[r] = Results[r]

In [6]:
Data.to_excel('NeaveExps.xlsx')

## Then calculate the clinopyroxene compositions predicted for lunar experiments

In [7]:
Data = pd.read_excel('Cpx-Data.xlsx', sheet_name = "Experiments")
Input = Data.iloc[:, np.where(Data.columns.str.contains('_Liq'))[0]]

In [8]:
Res_IW = M.equilibrate_multi(Model = "pMELTS", P_bar = Data['P(kbar)']*1000, T_C = Data['T(C)'], 
                          bulk = Input, fO2_buffer = "IW")

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
Results = pd.DataFrame(columns = Res_IW['0']['clinopyroxene1'].columns, 
                       data = np.zeros((15, len(Res_IW['0']['clinopyroxene1'].columns))))

T_C = np.zeros(len(Results['SiO2']))

for i in Res_IW:
    if 'clinopyroxene1' in list(Res_IW[i].keys()):
        Results.iloc[int(i), :] = Res_IW[i]['clinopyroxene1']
        
Results['FeOt'] = Results['FeO'] + Results['Fe2O3']*71.844/(159.69/2)
Results = Results.add_suffix('_MELTS_Cpx')    

for r in Results:
    Data[r] = Results[r]

In [10]:
Data.to_excel('Cpx-Data.xlsx', sheet_name = "Experiments")